In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
path = "/content/drive/MyDrive/Colab Notebooks"
os.chdir(path)

Mounted at /content/drive


In [2]:
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers,regularizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import math
import json
from sklearn.model_selection import train_test_split
import pandas as pd
import jieba
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.metrics import accuracy_score,f1_score
from tensorflow.keras.layers import concatenate
import numpy as np
import random

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as Data
import matplotlib.pyplot as plt

torch.set_default_tensor_type(torch.DoubleTensor)

SEED = 30

torch.manual_seed(SEED) 
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True 

embedding_size = 300
sequence_length = 50
num_classes = 2

In [4]:
stopwords=pd.read_csv("./data/CNENstopwords.txt", index_col=False,sep='\t',quoting=3,names=['stopwords'],encoding='utf-8')
stopwords = stopwords.stopwords.values.tolist()
blackword = ['\t','\n',' ','\xa0','\u200b']
for i in blackword:
    stopwords.append(i)

In [5]:
def drop_stopwords(line, stopwords):
    line_clean = []
    for word in line:
        if word in stopwords:
            continue
        line_clean.append(word)
    return line_clean

def cleantext(line):
    line = line.strip()
    line = re.sub(r'[^\w\s]','',line)
    current_seg = jieba.lcut(line)
    for i in range(current_seg.count(' ')):
        current_seg.remove(' ')
    for i in current_seg:
        if i == '\xa0':
            current_seg.remove(i)
    current_seg = drop_stopwords(current_seg, stopwords)
    return current_seg

def cleanurl(line):
    line = delgoogleweblight(line)
    blackurlparts = ['www', 'com', 'cn', 'http', 'https']
    comp = []
    for i in list(filter(bool, re.split('[^a-zA-Z]',line))):
        if i not in blackurlparts and len(i)>2:
            comp.append(i)
    return comp

def delgoogleweblight(url):
    a = url.strip().split('/')
    if a[2]=='googleweblight.com':
        a.remove('')
        a.remove('googleweblight.com')
        a.remove('fp?u='+a[0])
        return '/'.join(a)
    else:
        return url

In [6]:
df_white = pd.read_excel("./data/whitedn.xlsx",usecols=['dn'])
df_black = pd.read_excel("./data/blackdn.xlsx",usecols=['dn'])
whitednlist = df_white['dn'].values.tolist()
blackdnlist = df_black['dn'].values.tolist()
with open("./data/google_search/google_search_py_yj.json",'rb') as load_f:
     load_dict = json.load(load_f)
datayj = load_dict['RECORDS']
with open("./data/google_search/google_search_py_cj.json",'rb') as load_f:
     load_dict = json.load(load_f)
datacj = load_dict['RECORDS']
with open("./data/google_search/google_search_py_dm.json",'rb') as load_f:
     load_dict = json.load(load_f)
datadm = load_dict['RECORDS']
data = datayj + datacj + datadm

whiteurllist,blackurllist = [],[]
blackword=['.pdf','/paper','/article','baidu','bilibili','sohu','weibo','sina','youtube','google','taobao','douban','youdao','baike','wiki','pedia','researchgate',
             'amazon','twitter','qq','tecent','patent','worldwidescience','zhuanli','163','rhhz','d-nb','zhihu','zhidao','csdn','jianshu',
             'baijiahao','zhuanlan','blog','statesurveys','bartoc','jstor','Cited by','被引用','数据库',' OR ','…']
for i in data:
    matchObj = i['url_path'].split(' › ')
    if matchObj[0] in whitednlist:
        cnt = 0
        for j in blackword:
            if re.search(j, i['url'], re.I) is None:
                cnt = cnt + 1
        if cnt == len(blackword):
            whiteurllist.append(i['url'])
        else:
            blackurllist.append(i['url'])
    if matchObj[0] in blackdnlist:
        blackurllist.append(i['url'])
whiteurllist=list(set(whiteurllist))
blackurllist=list(set(blackurllist))
print(len(whiteurllist),len(blackurllist))

datadic_test = {'text':[], 'url':[], 'label':[]}
for i in data:
    if i['url'] in whiteurllist:
        datadic_test['text'].append(cleantext(i['title']+i['desc'].strip()))
        datadic_test['url'].append(cleanurl(i['url']))
        datadic_test['label'].append(1)
    elif i['url'] in blackurllist:
        datadic_test['text'].append(cleantext(i['title']+i['desc'].strip()))
        datadic_test['url'].append(cleanurl(i['url']))
        datadic_test['label'].append(0)
data_test = pd.DataFrame(datadic_test)
data_test

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...


9261 10901


Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.989 seconds.
DEBUG:jieba:Loading model cost 0.989 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


,text,url,label
0,"[SuperScript, SQLSSRS, MSDN2013, SQL, solution...","[social, msdn, microsoft, Forums, superscript,...",0
1,"[Face, recognition, unconstrained, videos, mat...","[researchgate, net, publication, Face, recogni...",0
2,"[Weak, Homogenization, Point, Processes, Space...","[researchgate, net, publication, Weak, Homogen...",0
3,"[PDF, Seawater, pH, reconstruction, boron, iso...","[researchgate, net, publication, Seawater, rec...",0
4,"[000114420416136443txt, SECgovSee, ldquoSupple...","[sec, gov, Archives, edgar, data, txt]",0
...,...,...,...
27411,"[Results, Salton, Seismic, Imaging, Project, S...","[iris, edu, webinar, results, from, the, salto...",1
27412,"[PDF, UTIGTR, 0083pdf, Institute, Geophysicsma...","[udc, utexas, edu, external, facilities, tech,...",0
27413,"[数据, 集, DataEase, 文档, 数据库, 数据, 集从, 数据源, 中, 选择,...","[dataease, docs, user, manual, dataset, config...",0
27414,"[数据, 集, Azure, Data, Factory, Azure, Synapse, ...","[docs, microsoft, azure, data, factory, concep...",0


In [7]:
def split_train_test_valid(df, train_rate, test_rate):
    random.seed(10)
    train_len = int(len(df) * train_rate)
    test_len = int(len(df) * test_rate)

    # split the dataframe
    idx = list(df.index)
    random.shuffle(idx)  # 将index列表打乱
    df_train = df.loc[idx[:train_len]]
    df_test = df.loc[idx[train_len:train_len+test_len]]
    df_valid = df.loc[idx[train_len+test_len:]]  # 剩下的就是valid
    
    text_train = df_train['text'].values.tolist()
    url_train = df_train['url'].values.tolist()
    y_train = df_train['label'].values.tolist()
    text_test = df_test['text'].values.tolist()
    url_test = df_test['url'].values.tolist()
    y_test = df_test['label'].values.tolist()
    text_valid = df_valid['text'].values.tolist()
    url_valid = df_valid['url'].values.tolist()
    y_valid = df_valid['label'].values.tolist()
    
    return text_train, url_train, y_train, text_test, url_test, y_test, text_valid ,url_valid, y_valid

In [8]:
text_train, url_train, y_train, text_test, url_test, y_test, text_valid ,url_valid, y_valid = split_train_test_valid(data_test, 0.6, 0.2)
origin_train_size = 7000
text_train_origin = text_train[:origin_train_size]
url_train_origin = url_train[:origin_train_size]
y_train_origin = y_train[:origin_train_size]
# print(y_train[0])
# f = open("./data/google_search/text_train_origin", "w")
# for line in text_train_origin:
#   for word in line:
#     f.write(word)
#     f.write("|")
#   f.write("\n")
# f.close()
# f = open("./data/google_search/url_train_origin", "w")
# for line in url_train_origin:
#   for word in line:
#     f.write(word)
#     f.write("|")
#   f.write("\n")
# f.close()
# f = open("./data/google_search/y_train_origin", "w")
# for line in y_train_origin:
#   f.write(str(line))
#   f.write("\n")
# f.close()

文本信息处理

In [9]:
text_train = []
text_uncertain = []
base_train_size = 4000
f = open("./data/google_search/text_train_origin", "r")
for i in range(base_train_size):
  text_train_str = f.readline()
  text_train_str = text_train_str[:-2]
  text_train_line = text_train_str.split("|")
  text_train.append(text_train_line)
for i in range(base_train_size, origin_train_size):
  text_uncertain_str = f.readline()
  text_uncertain_str = text_uncertain_str[:-2]
  text_uncertain_line = text_uncertain_str.split("|")
  text_uncertain.append(text_uncertain_line)
f.close()
url_train = []
url_uncertain = []
f = open("./data/google_search/url_train_origin", "r")
for i in range(base_train_size):
  url_train_str = f.readline()
  url_train_str = url_train_str[:-2]
  url_train_line = url_train_str.split("|")
  url_train.append(url_train_line)
for i in range(base_train_size, origin_train_size):
  url_uncertain_str = f.readline()
  url_uncertain_str = url_uncertain_str[:-2]
  url_uncertain_line = url_uncertain_str.split("|")
  url_uncertain.append(url_uncertain_line)
f.close()
y_train = []
y_uncertain = []
f = open("./data/google_search/y_train_origin", "r")
for i in range(base_train_size):
  y_train_str = f.readline()
  y_train_line = int(y_train_str)
  y_train.append(y_train_line)
for i in range(base_train_size, origin_train_size):
  y_uncertain_str = f.readline()
  y_uncertain_str = y_uncertain_str[:-2]
  y_uncertain_line = y_uncertain_str.split("|")
  y_uncertain.append(y_uncertain_line)
f.close()
print(len(y_uncertain))
print(y_train[0])

3000
1


In [10]:
max_num_words = 20000
tokenizer=Tokenizer(num_words=max_num_words,lower=True,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~� ',oov_token='<OOV>')
tokenizer.fit_on_texts(text_train)
vocab_text={word:i+1 for i, word in enumerate(tokenizer.word_index) if i+1<max_num_words}
len(vocab_text)

19999

In [11]:
from gensim.models import Word2Vec
shortmodel = Word2Vec(sentences=text_train, size=300, window=5, min_count=1, workers=4)
embedding_matrix_text = np.zeros((len(vocab_text) + 1, 300))
for word, i in vocab_text.items():
    try:
        embedding_vector = shortmodel.wv[str(word)]
        embedding_matrix_text[i] = embedding_vector
    except KeyError:
        continue
embedding_matrix_text = torch.from_numpy(embedding_matrix_text)
embedding_matrix_text.shape

torch.Size([20000, 300])

In [12]:
max_num_words = 8000
tokenizer=Tokenizer(num_words=max_num_words,lower=True,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~� ',oov_token='<OOV>')
tokenizer.fit_on_texts(url_train)
vocab_url={word:i+1 for i, word in enumerate(tokenizer.word_index) if i+1<max_num_words}
len(vocab_url)

6881

In [13]:
urlmodel = Word2Vec(sentences=url_train, size=300, window=5, min_count=1, workers=4)
embedding_matrix_url = np.zeros((len(vocab_url) + 1, 300))
for word, i in vocab_url.items():
    try:
        embedding_vector = urlmodel.wv[str(word)]
        embedding_matrix_url[i] = embedding_vector
    except KeyError:
        continue
embedding_matrix_url = torch.from_numpy(embedding_matrix_url)
embedding_matrix_url.shape

torch.Size([6882, 300])

In [14]:
embedding_matrix = torch.cat((embedding_matrix_text, embedding_matrix_url), 0)
embedding_matrix.shape

torch.Size([26882, 300])

In [15]:
text_sequence_length, url_sequence_length = 50, 8
x_train = np.concatenate([pad_sequences(tokenizer.texts_to_sequences(text_train), maxlen=text_sequence_length), pad_sequences(tokenizer.texts_to_sequences(url_train), maxlen=url_sequence_length)+20000],axis=1)
x_test = np.concatenate([pad_sequences(tokenizer.texts_to_sequences(text_test), maxlen=text_sequence_length), pad_sequences(tokenizer.texts_to_sequences(url_test), maxlen=url_sequence_length)+20000],axis=1)
x_valid = np.concatenate([pad_sequences(tokenizer.texts_to_sequences(text_valid), maxlen=text_sequence_length), pad_sequences(tokenizer.texts_to_sequences(url_valid), maxlen=url_sequence_length)+20000],axis=1)

In [16]:
class TextDataset(Data.Dataset):
    """LongText2000 dataset."""

    def __init__(self, data_root, data_label):
        self.data = data_root
        self.label = data_label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        data = self.data[index]
        labels = self.label[index]
        return data, labels

In [17]:
train_loader = Data.DataLoader(TextDataset(x_train, y_train), batch_size=100, shuffle=True, drop_last=False, num_workers=4, pin_memory=True)
validate_loader = Data.DataLoader(TextDataset(x_valid, y_valid), batch_size=100, shuffle=True, drop_last=False, num_workers=4, pin_memory=True)
test_loader = Data.DataLoader(TextDataset(x_test, y_test), batch_size=100, shuffle=True, drop_last=False, num_workers=4, pin_memory=True)
print(len(train_loader))
print(len(validate_loader))
print(len(test_loader))

40
55
55


In [18]:
class textCNN(nn.Module):
    def __init__(self):
        super(textCNN, self).__init__()
        Knum = 256 
        Ks = [3,4,5]
        
        self.embed = nn.Embedding(len(vocab_text)+len(vocab_url)+2, embedding_size, _weight=embedding_matrix) 
        
        self.convs = nn.ModuleList([nn.Conv2d(1,Knum,(K,embedding_size)) for K in Ks])
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(len(Ks)*Knum,2) 
        
    def forward(self,x):
        x = self.embed(x) #(N,W,D)
        
        x = x.unsqueeze(1) #(N,Ci,W,D)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs] # len(Ks)*(N,Knum,W)
        x = [F.max_pool1d(line,line.size(2)).squeeze(2) for line in x]  # len(Ks)*(N,Knum)
        
        x = torch.cat(x,1) #(N,Knum*len(Ks))
        
        x = self.dropout(x)
        logit = self.fc(x)
        return logit

if hasattr(torch.cuda, 'empty_cache'):
	  torch.cuda.empty_cache()

In [19]:
EPOCH = 20
model = textCNN().cuda()
loss_function = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

lossdata, trainaccdata, validaccdata = [], [], []
best_valid_loss = float('inf')


In [ ]:
for epoch in range(EPOCH):
    epoch_loss = 0
    epoch_acc = 0
    total_len = 0
    model.train()
    for i, (data, labels) in enumerate(train_loader):
        b_x = torch.Tensor(data.double()).long()
        b_y = torch.Tensor(labels.double()).long()
        #forward
        if torch.cuda.is_available():
            b_x, b_y = b_x.cuda(), b_y.cuda()
        output = model(b_x)
        #loss
        loss = loss_function(output, b_y)
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pred_y = torch.max(output, 1)[1].data.squeeze()
        acc = (b_y == pred_y)
        acc = acc.cpu().numpy().sum()
        accuracy = acc / (b_y.size(0))
        
        epoch_loss += loss.item()
        epoch_acc += accuracy
        total_len += 1
        print("iter:", str(i), "  loss:", loss.item(), "  accuracy:", accuracy)
    
    trainloss = epoch_loss/total_len
    trainaccuracy = epoch_acc/total_len
    lossdata.append(trainloss)
    trainaccdata.append(trainaccuracy)
    print("**train result**   epoch:"+str(epoch)+"  loss:"+str(trainloss)+"  accuracy:"+str(trainaccuracy))
    
    epoch_valid_loss = 0
    epoch_valid_acc = 0
    total_valid_len = 0
    model.eval()
    with torch.no_grad():
        for i, (data, labels) in enumerate(validate_loader):
            b_x = torch.Tensor(data.double()).long()
            b_y = torch.Tensor(labels.double()).long()
            if torch.cuda.is_available():
                b_x, b_y = b_x.cuda(), b_y.cuda()
            valid_output = model(b_x)
            loss = loss_function(valid_output, b_y)
            pred_y = torch.max(valid_output, 1)[1].data.squeeze()
            acc = (pred_y == b_y)
            acc = acc.cpu().numpy().sum()
            accuracy = acc / (b_y.size(0))

            epoch_valid_loss = epoch_valid_loss + loss.item()
            epoch_valid_acc = epoch_valid_acc + accuracy
            total_valid_len += 1
    
    valid_loss = epoch_valid_loss/total_valid_len
    valid_acc = epoch_valid_acc/total_valid_len
    validaccdata.append(valid_acc)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'CNN-model-short-0413.pt')

    print("**validation result**   epoch:"+str(epoch)+"  loss:"+str(valid_loss)+
          "  accuracy:"+str(epoch_valid_acc/total_valid_len))

iter: 0   loss: 0.6945804686196715   accuracy: 0.48
iter: 1   loss: 0.6807671205811475   accuracy: 0.58
iter: 2   loss: 0.6845548471426881   accuracy: 0.57
iter: 3   loss: 0.6834462518071868   accuracy: 0.53
iter: 4   loss: 0.6805103987358385   accuracy: 0.51
iter: 5   loss: 0.6783402619773932   accuracy: 0.63
iter: 6   loss: 0.6728349617788715   accuracy: 0.67
iter: 7   loss: 0.6732721003519111   accuracy: 0.56
iter: 8   loss: 0.6447896136525426   accuracy: 0.69


In [ ]:
plt.plot(trainaccdata)
plt.plot(validaccdata)
plt.plot(lossdata)
plt.legend(['train-accuracy', 'valid-accuracy', 'loss'], loc='upper left')
plt.show()

In [ ]:
model.load_state_dict(torch.load('CNN-model-short-0413.pt'))
model.eval()

test_loss = 0
test_acc = 0
test_len = 0
with torch.no_grad():
    for i, (data, labels) in enumerate(test_loader):
        test_x = torch.Tensor(data.double()).long()
        test_y = torch.Tensor(labels.double()).long()
        if torch.cuda.is_available():
            test_x, test_y = test_x.cuda(), test_y.cuda()
        test_output = model(test_x)
        loss = loss_function(test_output, test_y)
        pred_y = torch.max(test_output, 1)[1].data.squeeze()
        acc = (pred_y == test_y)
        acc = acc.cpu().numpy().sum()
        accuracy = acc / (test_y.size(0))
        
        test_loss +=  loss.item()
        test_acc += accuracy
        test_len += 1

print("**Test result**  loss:"+str(test_loss/test_len)+"  accuracy:"+str(test_acc/test_len))

In [ ]:
df_annotation = pd.read_excel('./data/标注网站（短文本）.xlsx')
url = list(df_annotation['url'].apply(cleanurl))
text = list(df_annotation['text'].apply(cleantext))
data_test = np.concatenate([pad_sequences(tokenizer.texts_to_sequences(text), maxlen=text_sequence_length), pad_sequences(tokenizer.texts_to_sequences(url), maxlen=url_sequence_length)+20000],axis=1)

In [ ]:
model = textCNN().cuda()
model.load_state_dict(torch.load('CNN-model-short-0413.pt'))
model.eval()
with torch.no_grad():
  xdata = torch.Tensor(data_test).long().cuda()
  test_output = model(xdata)
  pred_y = torch.max(test_output, 1)[1].data.squeeze()
  np.set_printoptions(suppress=True)
  np.set_printoptions(precision=4)
  print(test_output.cpu().numpy())
  print(pred_y)
  for i in pred_y:
    print(i.item())

In [ ]:
import torch
from torch import nn

def normalization(p, n):
    unnormalized_pred = torch.tensor([p, n])
    normalized_pred = nn.Softmax(dim=0)(unnormalized_pred)
    return normalized_pred


def least_confidence(p, n):
    tensor = normalization(p, n)
    p = tensor[0]
    q = tensor[1]
    if p > q:
        return p
    else:
        return q


def entropy_reduction(p, n):
    tensor = normalization(p, n)
    tensor = tensor * torch.log(tensor)
    print("tensor = ", tensor)
    p = tensor[0]
    q = tensor[1]
    return p + q
    #之后取最小的为最需要标注的，故此处为+


def margin_sampling(p, n):
    tensor = normalization(p, n)
    print("tensor = ", tensor)
    p = tensor[0]
    q = tensor[1]
    if p > q:
        return p - q
    else:
        return q - p


def uncertainty(p, n, uncertainty):
    if uncertainty == "least_confidence":
      return least_confidence(p, n)
    elif uncertainty == "entropy_reduction":
      return entropy_reduction(p, n)
    elif uncertainty == "margin_sampling":
      return margin_sampling(p, n)
    else:
      return (p, n)

In [ ]:
data_test = np.concatenate([pad_sequences(tokenizer.texts_to_sequences(text_uncertain), maxlen=text_sequence_length), pad_sequences(tokenizer.texts_to_sequences(url_uncertain), maxlen=url_sequence_length)+20000],axis=1)
model = textCNN().cuda()
model.load_state_dict(torch.load('CNN-model-short-0413.pt'))
model.eval()
with torch.no_grad():
  xdata = torch.Tensor(data_test).long().cuda()
  test_output = model(xdata)
  print(test_output.shape)
  pred_y = torch.max(test_output, 1)[1].data.squeeze()
  np.set_printoptions(suppress=True)
  np.set_printoptions(precision=4)
  print(test_output.cpu().numpy())
  print(pred_y)
  print(pred_y.shape)
uncertainty_dict = dict()
for i in range(origin_train_size - base_train_size):
  uncertainty_result = uncertainty(test_output[i][0], test_output[i][1], "margin_sampling")
  uncertainty_dict[base_train_size + i] = uncertainty_result

uncertainty_new = 1000
uncertainty_list = sorted(uncertainty_dict.items(), key=lambda x: x[1], reverse=False)
uncertainty_list = uncertainty_list[:uncertainty_new]
uncertainty_line = []
for i in range(uncertainty_new):
  uncertainty_line.append(uncertainty_list[i][0])
print(uncertainty_line)

In [ ]:
# f = open("./data/google_search/text_train_uncertainty3", "w")
# for line_number, line in enumerate(text_train_origin):
#   if uncertainty_line.count(line_number):
#     for word in line:
#       f.write(word)
#       f.write("|")
#     f.write("\n")
# f.close()
# f = open("./data/google_search/url_train_uncertainty3", "w")
# for line_number, line in enumerate(url_train_origin):
#   if uncertainty_line.count(line_number):
#     for word in line:
#       f.write(word)
#       f.write("|")
#     f.write("\n")
# f.close()
# f = open("./data/google_search/y_train_uncertainty3", "w")
# for line_number, line in enumerate(y_train_origin):
#   if uncertainty_line.count(line_number):
#     f.write(str(line))
#     f.write("\n")
# f.close()

In [ ]:
import random

random_line = []
random_count = 0;
while random_count < 1000:
  random_num = random.randint(4000, 6999)
  if random_line.count(random_num) == 0:
    random_line.append(random_num)
    random_count += 1

f = open("./data/google_search/text_train_random", "w")
for line_number, line in enumerate(text_train_origin):
  if random_line.count(line_number):
    for word in line:
      f.write(word)
      f.write("|")
    f.write("\n")
f.close()
f = open("./data/google_search/url_train_random", "w")
for line_number, line in enumerate(url_train_origin):
  if random_line.count(line_number):
    for word in line:
      f.write(word)
      f.write("|")
    f.write("\n")
f.close()
f = open("./data/google_search/y_train_random", "w")
for line_number, line in enumerate(y_train_origin):
  if random_line.count(line_number):
    f.write(str(line))
    f.write("\n")
f.close()

In [ ]:
text_tune = []
f = open("./data/google_search/text_train_random", "r")
for i in range(uncertainty_new):
  text_tune_str = f.readline()
  text_tune_str = text_tune_str[:-2]
  text_tune_line = text_tune_str.split("|")
  text_tune.append(text_tune_line)
f.close()
url_tune = []
f = open("./data/google_search/url_train_random", "r")
for i in range(uncertainty_new):
  url_tune_str = f.readline()
  url_tune_str = url_tune_str[:-2]
  url_tune_line = url_tune_str.split("|")
  url_tune.append(url_tune_line)
f.close()
y_tune = []
f = open("./data/google_search/y_train_random", "r")
for i in range(uncertainty_new):
  y_tune_str = f.readline()
  y_tune_line = int(y_tune_str)
  y_tune.append(y_tune_line)
f.close()
print(len(y_tune), len(text_tune), len(url_tune))
print(y_tune[0])


x_tune = np.concatenate([pad_sequences(tokenizer.texts_to_sequences(text_tune), maxlen=text_sequence_length), pad_sequences(tokenizer.texts_to_sequences(url_tune), maxlen=url_sequence_length)+20000],axis=1)
tune_loader = Data.DataLoader(TextDataset(x_tune, y_tune), batch_size=100, shuffle=True, drop_last=False, num_workers=4, pin_memory=True)


In [ ]:
model = textCNN().cuda()
model.load_state_dict(torch.load('CNN-model-short-0413.pt'))

epoch_tune = 5

for name, para in model.named_parameters():
  print(name)
  if name != 'fc.weight' and name != 'fc.bias':
    para.requires_grad=False
  print(para.requires_grad)  

In [ ]:
optimizer = optim.Adam(filter(lambda p:p.requires_grad,model.parameters()), lr=1e-3)

for epoch in range(epoch_tune):
    epoch_loss = 0
    epoch_acc = 0
    total_len = 0
    model.train()
    for i, (data, labels) in enumerate(tune_loader):
        b_x = torch.Tensor(data.double()).long()
        b_y = torch.Tensor(labels.double()).long()
        #forward
        if torch.cuda.is_available():
            b_x, b_y = b_x.cuda(), b_y.cuda()
        output = model(b_x)
        #loss
        loss = loss_function(output, b_y)
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pred_y = torch.max(output, 1)[1].data.squeeze()
        acc = (b_y == pred_y)
        acc = acc.cpu().numpy().sum()
        accuracy = acc / (b_y.size(0))
        
        epoch_loss += loss.item()
        epoch_acc += accuracy
        total_len += 1
        print("iter:", str(i), "  loss:", loss.item(), "  accuracy:", accuracy)
    
    trainloss = epoch_loss/total_len
    trainaccuracy = epoch_acc/total_len
    lossdata.append(trainloss)
    trainaccdata.append(trainaccuracy)
    print("**train result**   epoch:"+str(epoch)+"  loss:"+str(trainloss)+"  accuracy:"+str(trainaccuracy))
    
    epoch_valid_loss = 0
    epoch_valid_acc = 0
    total_valid_len = 0
    model.eval()
    with torch.no_grad():
        for i, (data, labels) in enumerate(validate_loader):
            b_x = torch.Tensor(data.double()).long()
            b_y = torch.Tensor(labels.double()).long()
            if torch.cuda.is_available():
                b_x, b_y = b_x.cuda(), b_y.cuda()
            valid_output = model(b_x)
            loss = loss_function(valid_output, b_y)
            pred_y = torch.max(valid_output, 1)[1].data.squeeze()
            acc = (pred_y == b_y)
            acc = acc.cpu().numpy().sum()
            accuracy = acc / (b_y.size(0))

            epoch_valid_loss = epoch_valid_loss + loss.item()
            epoch_valid_acc = epoch_valid_acc + accuracy
            total_valid_len += 1
    
    valid_loss = epoch_valid_loss/total_valid_len
    valid_acc = epoch_valid_acc/total_valid_len
    validaccdata.append(valid_acc)
    

    print("**validation result**   epoch:"+str(epoch)+"  loss:"+str(valid_loss)+
          "  accuracy:"+str(epoch_valid_acc/total_valid_len))

In [ ]:
df_annotation = pd.read_excel('./data/标注网站（短文本）.xlsx')
url = list(df_annotation['url'].apply(cleanurl))
text = list(df_annotation['text'].apply(cleantext))
data_test = np.concatenate([pad_sequences(tokenizer.texts_to_sequences(text), maxlen=text_sequence_length), pad_sequences(tokenizer.texts_to_sequences(url), maxlen=url_sequence_length)+20000],axis=1)
model = textCNN().cuda()
model.load_state_dict(torch.load('CNN-model-short-0413.pt'))
model.eval()
with torch.no_grad():
  xdata = torch.Tensor(data_test).long().cuda()
  test_output = model(xdata)
  pred_y = torch.max(test_output, 1)[1].data.squeeze()
  np.set_printoptions(suppress=True)
  np.set_printoptions(precision=4)
  print(test_output.cpu().numpy())
  print(pred_y)
  for i in pred_y:
    print(i.item())

In [ ]:
df_annotation = pd.read_excel('./data/test_专家标注.xlsx')
url = list(df_annotation['url'].apply(cleanurl))
text = list(df_annotation['text'].apply(cleantext))
data_test = np.concatenate([pad_sequences(tokenizer.texts_to_sequences(text), maxlen=text_sequence_length), pad_sequences(tokenizer.texts_to_sequences(url), maxlen=url_sequence_length)+20000],axis=1)

In [ ]:
model = textCNN().cuda()
model.load_state_dict(torch.load('CNN-model-short-0413.pt'))
model.eval()
with torch.no_grad():
  xdata = torch.Tensor(data_test).long().cuda()
  test_output = model(xdata)
  pred_y = torch.max(test_output, 1)[1].data.squeeze()
  np.set_printoptions(suppress=True)
  np.set_printoptions(precision=4)
  print(test_output.cpu().numpy())
  print(pred_y)

[[ 0.8572 -1.5965]
 [ 3.3726 -4.7692]
 [-1.96    1.3772]
 [ 0.1851 -0.5574]
 [-2.1963  1.4971]
 [-0.5852  0.6297]
 [-0.4383 -0.3263]
 [-2.521   2.0062]
 [-0.0932 -0.3486]
 [-0.9974  0.8558]
 [-1.3429  1.2821]
 [-1.3155  0.711 ]
 [ 0.4079 -1.4928]
 [-3.9044  2.9942]
 [-0.3913 -0.0354]
 [-1.0962  0.6422]
 [-3.1715  2.4153]
 [-4.0781  3.2668]
 [ 0.2179 -1.0006]
 [ 1.707  -3.2022]
 [ 2.1403 -2.6483]
 [ 2.4852 -3.7152]
 [-1.0268 -0.1236]
 [ 0.8063 -1.2209]
 [-0.9913  0.9059]
 [ 3.07   -4.1469]
 [-2.0972  0.9282]
 [-0.4184  0.1879]
 [-0.1747 -0.1564]
 [-2.0159  1.9254]
 [-0.415   0.0411]
 [-0.5729 -0.4185]
 [ 1.663  -3.1132]
 [ 1.3453 -1.854 ]
 [-0.3716 -0.7024]
 [ 1.2584 -2.1513]
 [ 0.9656 -2.1928]
 [ 2.9844 -4.4559]
 [-0.5287  0.5289]
 [-0.7447  0.8353]
 [ 0.9264 -1.5305]
 [ 2.3951 -2.2797]
 [-1.7441  1.2047]
 [ 3.8032 -5.017 ]
 [-2.657   2.2112]
 [-1.9695  0.9624]
 [-3.323   2.9441]
 [-0.6805  0.1759]
 [ 0.3442 -1.1107]
 [ 0.7594 -1.471 ]
 [ 0.0581 -1.7733]
 [-0.7965  0.118 ]
 [-0.6135 -0